# TSA Chapter 10: Unemployment Rate Stationarity Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10_unemployment_stationarity/TSA_ch10_unemployment_stationarity.ipynb)

This notebook demonstrates:
- Testing the stationarity of U.S. unemployment rate via ADF test and comparing original vs differenced series.

In [ ]:
!pip install pandas-datareader statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import pandas_datareader as pdr
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download unemployment data
unemp = pdr.get_data_fred('UNRATE', start='2010-01-01', end='2025-01-15')
unemp_series = unemp['UNRATE']
print(f'Data: {len(unemp_series)} observations')

# 70% / 20% / 10% split
train_end = '2020-06-01'
val_start = '2020-07-01'
val_end = '2023-06-01'
test_start = '2023-07-01'

train_data = unemp_series[unemp_series.index <= train_end]
val_data = unemp_series[(unemp_series.index >= val_start) & (unemp_series.index <= val_end)]
test_data = unemp_series[unemp_series.index >= test_start]
train_val_data = unemp_series[unemp_series.index <= val_end]

print(f'Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}')

# Stationarity analysis
from statsmodels.tsa.stattools import adfuller

adf_orig = adfuller(train_data.dropna())
train_diff = train_data.diff().dropna()
adf_diff = adfuller(train_diff)
print(f'Original: ADF p = {adf_orig[1]:.4f}')
print(f'Differenced: ADF p = {adf_diff[1]:.4f}')

fig, axes = plt.subplots(2, 2, figsize=(10, 5))

axes[0,0].plot(train_data.index, train_data.values, color=COLORS['blue'], lw=1.2, label='Train')
axes[0,0].plot(val_data.index, val_data.values, color=COLORS['purple'], lw=1.2, label='Val')
axes[0,0].plot(test_data.index, test_data.values, color=COLORS['green'], lw=1.2, label='Test')
axes[0,0].set_title('Original Series', fontweight='bold')
color = COLORS['red'] if adf_orig[1] > 0.05 else COLORS['green']
axes[0,0].text(0.02, 0.95, f'ADF p = {adf_orig[1]:.3f}', transform=axes[0,0].transAxes, fontsize=9, va='top', color=color)

axes[0,1].plot(train_diff.index, train_diff.values, color=COLORS['blue'], lw=1)
axes[0,1].axhline(0, color='black', lw=0.5, alpha=0.3)
axes[0,1].set_title('First Difference $\\Delta y_t$', fontweight='bold')
axes[0,1].text(0.02, 0.95, f'ADF p < 0.001\n(Stationary)', transform=axes[0,1].transAxes, fontsize=9, va='top', color=COLORS['green'])

plot_acf(train_data.values, ax=axes[1,0], lags=20, alpha=0.05)
axes[1,0].set_title('ACF: Original (slow decay)', fontweight='bold')

plot_acf(train_diff.values, ax=axes[1,1], lags=20, alpha=0.05)
axes[1,1].set_title('ACF: Differenced', fontweight='bold')

fig.tight_layout()
save_chart(fig, 'unemployment_stationarity')
plt.show()